In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

tf.reset_default_graph()  #tensorflow 그래프 리셋

#1. data loading
data=pd.read_csv("./data/digit/train.csv",sep=",")
data_2=pd.read_csv("./data/digit/test.csv",sep=",")
data_2=np.array(data_2.values, dtype=np.float32)
#data_2=np.float32(data_2)

sess=tf.Session()
num_data=int(data.shape[0]*0.7)
data_train=data.loc[0:num_data,:]
data_test=data.loc[num_data:,:]

train_x=np.array(data_train.iloc[:,1:785].values)
train_y=sess.run(tf.one_hot(data_train.iloc[:,0],10)) 

test_x=np.array(data_test.iloc[:,1:785].values)
test_y=sess.run(tf.one_hot(data_test.iloc[:,0],10)) 

# #2. placeholder
X=tf.placeholder(shape=[None,784], dtype=tf.float32)
Y=tf.placeholder(shape=[None,10], dtype=tf.float32)
keep_rate = tf.placeholder(dtype=tf.float32)

X_img = tf.reshape(X, shape=[-1,28,28,1]) #데이터의 개수 : -1은계산을 맡김
#filter 생성
W1=tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))

#convolution
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding="SAME")
#relu
L1=tf.nn.relu(L1)
#max pooling
L1= tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")


#3.2 convolution layer2
L2=tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu)
L2=tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2], padding="SAME", strides=2)
#convolution layer 끝


#4.FC(Neural Network)
L2=tf.reshape(L2, shape=[-1, 7*7*64]) 

#5.weight & bias
W2=tf.get_variable("weight2", shape=[7*7*64,256]
                   , initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.random_normal([256]),name="bias2")
_layer1=tf.nn.relu(tf.matmul(L2,W2)+b2)
layer1=tf.nn.dropout(_layer1, keep_prob=keep_rate)

W3=tf.get_variable("weight3", shape=[256,256]
                   , initializer=tf.contrib.layers.xavier_initializer())
b3=tf.Variable(tf.random_normal([256]),name="bias3")
_layer2=tf.nn.relu(tf.matmul(layer1,W3)+b3)
layer2=tf.nn.dropout(_layer2, keep_prob=keep_rate)   #keep_prob : (여기서는 keep_rate를 0.5로 지정해줌) 50%의 뉴론이 drop out되도록
                                                     #           만약 0.7이면 30%의 뉴론이 drop out

W4=tf.get_variable("weight4", shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
b4=tf.Variable(tf.random_normal([10]),name="bias4")

#Hypothesis
H=tf.matmul(layer2,W4)+b4

#Cost Fuction
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=H,labels=Y))

#train
train=tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

#session, 초기화
#sess=tf.Session()
sess.run(tf.global_variables_initializer())

#학습
train_epoch=1
batch_size=100

for step in range(train_epoch):
    num_of_iter = int(train_x.shape[0] / batch_size)  #반복횟수 : 전체데이터 / batch_size
    for i in range(num_of_iter):
        batch_x, batch_y = train_x[i*batch_size:i*batch_size+batch_size]
        ,train_y[i*batch_size:i*batch_size+batch_size]  
        _, cost_val= sess.run([train, cost],feed_dict={X:batch_x,Y:batch_y,keep_rate:0.5}) 
        #batch_x, batch_y = data_x[i*batch_size:i*batch_size+batch_size],data_y[i*batch_size:i*batch_size+batch_size]      #전체의 훈련데이터(mnist.train)에서 데이터를 100개씩 가져옴
        
        
    print("cost: {}".format(cost_val))

#정확도 측정
predict=tf.argmax(H,1)
correct=tf.equal(predict, tf.argmax(Y,1))
accuracy=tf.reduce_sum(tf.cast(correct, dtype=tf.float32))
result_sum=0
num_of_iter = int(test_x.shape[0]/batch_size)
    
for i in range(num_of_iter):
    batch_x, batch_y = test_x[i*batch_size:i*batch_size+batch_size],test_y[i*batch_size:i*batch_size+batch_size] 
    correct_num=sess.run(accuracy, feed_dict={X:batch_x,Y:batch_y,keep_rate:1})
    result_sum+=correct_num
    
print("정확도 : {}".format(result_sum/test_x.shape[0]))
# final=sess.run(predict, feed_dict={X:data_2, keep_rate:1 })
# print("예측값",final)  

cost: 0.15663830935955048
정확도 : 0.9730973732243473


In [8]:
batch_size=100
num_of_iter = int(data_2.shape[0]/batch_size)
final=[]
for i in range(num_of_iter):
    batch_x= data_2[i*batch_size:i*batch_size+batch_size] 
    #print("예측값",sess.run(predict, feed_dict={X:batch_x, keep_rate:1}))
    values=sess.run(predict, feed_dict={X:batch_x, keep_rate:1})
    final.append(values.tolist())
final=np.array(final)
final=np.ravel(final)
final

array([2, 0, 9, ..., 3, 9, 2])

In [144]:
batch_size=100
num_of_iter = int(data_2.shape[0]/batch_size)
final=[]
for i in range(num_of_iter):
    batch_x= data_2[i*batch_size:i*batch_size+batch_size] 
    #print("예측값",sess.run(predict, feed_dict={X:batch_x, keep_rate:1}))
    values=sess.run(predict, feed_dict={X:batch_x, keep_rate:1})
    final.append(values.tolist())
    

final=np.array(final)
final=np.ravel(final)
len(final)

28000

In [145]:
import tensorflow as tf
import numpy as np
import pandas as pd

data_3=pd.read_csv("./data/digit/sample_submission.csv",sep=",")

data_3["Label"]=final

data_3.to_csv("Digit_B", index=False)

- train data를 이용해서 해당 모델을 학습
- 최종적으로 얻어내는 것은 Hypothesis
- H를 이용해서 예측값 도출
- test data(입력 feature)가 들어감
- H=>[0.342, 0,234 ...]    가설 값 : 이 그림에 대한 픽셀값(10개로 구성, 다 더하면 1)을 모델을 통해 돌려보니 나오는 예측값(=확률값)
- 이렇게 나온 값(H)에 argmax를 취하면 가장 큰 값의 index를 얻어낼 수 있고 이 값을 입력 label의 argmax와 비교해서 
- 만약 같으면 예측이 잘 되었다. 틀리면 예측 모델이 이상함

- ensemble(앙상블)
- 모델이 여러개 ex10개
- 각각의 모델의 학습시킴
- 각 모델의 입력 파라미터(이미지 픽셀)을 넣어서 예측값을 알아냄
- Model1=>[0.342, 0,234 ...]  Model1=>[0.234, 0,64 ...]   Model1=>[0.52, 0,29 ...] ...model10
- 각 열을 sum

- self=현재 자기자신을 가르키는 인스턴스
- self.name = 이 클래스의 객체가 가지는 변수
- 클래스 안에 정의되어 있는 함수의 인자 첫번째는 self를 줘야한다

In [ ]:
import tensorflow as tf
class Student:
    def __init__(self, name, kor, eng, math):
        self.name=name
        self.kor=kor
        self.eng=eng
        self.math=math
        
    def calc_avg(self):
        return(self.kor+self.eng+self.math)/3
    
stu1=Student("홍길동", 10,20,30)
print(stu1.calc_avg())
stud2=Student("김길동", 30,40,50)


In [ ]:
import tensorflow as tf
#여러개의 모델을 만들어서 관리를 해야 함
#각각의 모델을 class의 instance로 만들어 관리
#우리가 만들 model의 데이터와 기능을 생각해서 class를 디자인

class CNNModel:
    #사용하는 데이터를 field(변수로 선언)
    #사용하는 기능은 함수(method)로 표현
    #-데이터를 로딩하는 기능은 넣지 않는다 => 공용으로 사용하기 때문(메모리 낭비가 심하다)
    #-tensorflow graph를 그리는 기능(모델을 구축)
    #-학습하는 기능
    #-정확도 측정
    #-예측작업
    def __init__(self, name, sess):
        self.name=name
        self.sess=sess
    def build_model(self):
        #placeholder부터 코드 작성(~train)
        #self.train=tf.train.Adamop~
    def exec_train(self):
        #sess.run([self.train,cost], feed_dict={})
        

#1. 데이터 로딩
#-판다스를 이용해서 데이터를 읽음
#2. 모델객체를 생성(10개)
#-sess=tf.Session()
mode=Model("model1",sess)
modle1.build_model()
model1.exec_train(x_data)
        

In [149]:
import tensorflow as tf
import numpy as np
import pandas as pd

tf.reset_default_graph()  #tensorflow 그래프 리셋

#1. data loading
data=pd.read_csv("./data/digit/train.csv",sep=",")
data_2=pd.read_csv("./data/digit/test.csv",sep=",")
data_2=np.array(data_2.values, dtype=np.float32)
#data_2=np.float32(data_2)

sess=tf.Session()
num_data=int(data.shape[0]*0.7)
data_train=data.loc[0:num_data,:]
data_test=data.loc[num_data:,:]

train_x=np.array(data_train.iloc[:,1:785].values)
train_y=sess.run(tf.one_hot(data_train.iloc[:,0],10)) 

test_x=np.array(data_test.iloc[:,1:785].values)
test_y=sess.run(tf.one_hot(data_test.iloc[:,0],10)) 

# #2. placeholder
X=tf.placeholder(shape=[None,784], dtype=tf.float32)
Y=tf.placeholder(shape=[None,10], dtype=tf.float32)
keep_rate = tf.placeholder(dtype=tf.float32)

X_img = tf.reshape(X, shape=[-1,28,28,1]) #데이터의 개수 : -1은계산을 맡김
#filter 생성
W1=tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))

#convolution
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding="SAME")
#relu
L1=tf.nn.relu(L1)
#max pooling
L1= tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")


#3.2 convolution layer2
L2=tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu)
L2=tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2], padding="SAME", strides=2)
#convolution layer 끝


#4.FC(Neural Network)
L2=tf.reshape(L2, shape=[-1, 7*7*64]) 

#5.weight & bias
W2=tf.get_variable("weight2", shape=[7*7*64,256], initializer=tf.contrib.layers.xavier_initializer())
b2=tf.Variable(tf.random_normal([256]),name="bias2")
_layer1=tf.nn.relu(tf.matmul(L2,W2)+b2)
layer1=tf.nn.dropout(_layer1, keep_prob=keep_rate)

W3=tf.get_variable("weight3", shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
b3=tf.Variable(tf.random_normal([256]),name="bias3")
_layer2=tf.nn.relu(tf.matmul(layer1,W3)+b3)
layer2=tf.nn.dropout(_layer2, keep_prob=keep_rate)   #keep_prob : (여기서는 keep_rate를 0.5로 지정해줌) 50%의 뉴론이 drop out되도록
                                                     #           만약 0.7이면 30%의 뉴론이 drop out

W4=tf.get_variable("weight4", shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
b4=tf.Variable(tf.random_normal([10]),name="bias4")

#Hypothesis
H=tf.matmul(layer2,W4)+b4

#Cost Fuction
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=H,labels=Y))

#train
train=tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

#session, 초기화
#sess=tf.Session()
sess.run(tf.global_variables_initializer())

#학습
train_epoch=1
batch_size=100

for step in range(train_epoch):
    num_of_iter = int(train_x.shape[0] / batch_size)  #반복횟수 : 전체데이터 / batch_size
    for i in range(num_of_iter):
        batch_x, batch_y = train_x[i*batch_size:i*batch_size+batch_size],train_y[i*batch_size:i*batch_size+batch_size]  
        _, cost_val= sess.run([train, cost],feed_dict={X:batch_x,Y:batch_y,keep_rate:0.5}) 
        #batch_x, batch_y = data_x[i*batch_size:i*batch_size+batch_size],data_y[i*batch_size:i*batch_size+batch_size]      #전체의 훈련데이터(mnist.train)에서 데이터를 100개씩 가져옴
        
        
    print("cost: {}".format(cost_val))

#정확도 측정
predict=tf.argmax(H,1)
correct=tf.equal(predict, tf.argmax(Y,1))
accuracy=tf.reduce_sum(tf.cast(correct, dtype=tf.float32))
result_sum=0
num_of_iter = int(test_x.shape[0]/batch_size)
    
for i in range(num_of_iter):
    batch_x, batch_y = test_x[i*batch_size:i*batch_size+batch_size],test_y[i*batch_size:i*batch_size+batch_size] 
    correct_num=sess.run(accuracy, feed_dict={X:batch_x,Y:batch_y,keep_rate:1})
    result_sum+=correct_num
    
print("정확도 : {}".format(result_sum/test_x.shape[0]))
print(H)

cost: 0.10792142152786255
정확도 : 0.9720657090707087
Tensor("add_2:0", shape=(?, 10), dtype=float32)


In [ ]:
test_x=np.array(data_test.iloc[:,1:785].values)
